In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-07-15 05:23:16--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6032 (5.9K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   5.89K  --.-KB/s    in 0s      

2025-07-15 05:23:17 (42.5 MB/s) - ‘minsearch.py.4’ saved [6032/6032]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll'

In [9]:
index.fit(documents)

In [10]:
boost = {'question' : 1.0, 'section': 1.0}

results = index.search(
    query=q,
    #filter_dict={'course':'llm-zoomcamp'},
    boost_dict=boost,
    num_results=5    
)    

In [11]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequen

In [12]:
#from openai import OpenAI
#client = OpenAI()
from google import genai
client = genai.Client()

In [13]:
from google.genai import types

In [14]:
response =  client.models.generate_content(
    model="gemini-2.5-flash",
    contents = q,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

It depends on the institution and the specific course. Some courses allow late enrollment, while others have strict deadlines due to the nature of the material or capacity limits.

Here's what you should do to find out:

1.  **Contact the Admissions Office or Registrar's Office:** They are the best source for official information.
2.  **Contact the Department or Instructor:** For some courses, the instructor might have discretion to allow late adds, especially if you can catch up quickly.
3.  **Check the Course Catalog or Website:** Sometimes, the policy on late enrollment is published there.

When you contact them, be prepared to ask:

*   "Is late enrollment still possible for [Course Name/Number]?"
*   "What is the last day to add this course?"
*   "Are there any special procedures or approvals needed for late enrollment?"
*   "What are the implications for missed classes or assignments?"

Good luck!


In [15]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make predictions or decisions.


In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Restrict your answers to the QUESTION to the content in CONTEXT only.

QUESTION : {question}

CONTEXT:
{context}

"""

In [17]:
context = ""
for doc in results:
    content = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [18]:
prompt = prompt_template.format(question=q, context=content).strip()

In [19]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Restrict your answers to the QUESTION to the content in CONTEXT only.

QUESTION : the course has already started, can I still enroll

CONTEXT:
section: General course-related questions
question: Course - Can I get support if I take the course in the self-paced mode?
answer: Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.
You can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.


In [20]:
response =  client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

Based on the provided context, there is no information available to answer whether you can still enroll if the course has already started. The context only discusses support for self-paced mode.


In [21]:
def search(query):
    boost = {'question' : 3.0, 'section': 0.5}

    results = index.search(
        query=q,
        #filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5    
    )
    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Restrict your answers to the QUESTION to the content in CONTEXT only.

QUESTION : {question}

CONTEXT:
{context}

""".strip()
    
    context = ""
    for doc in search_results:
        content = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=content).strip()
    return prompt

In [23]:
def llm(prompt):
    client = genai.Client()
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents= prompt,
    )
    
    return response.text

In [24]:
query = 'how do I run kafka?'
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
rag(query)

'The context does not provide information on how to run Kafka.'

In [26]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [27]:
# elsatic search saves data on disc
from elasticsearch import Elasticsearch

In [28]:
es_client = Elasticsearch('http://localhost:9200')

In [30]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [31]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████| 948/948 [00:04<00:00, 236.00it/s]


In [33]:
query = 'I just discovered the course, can I still join it?'

In [40]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [41]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [42]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [43]:
rag(query)

'Yes, you can still join. You can just start learning and submitting homework without registering. It is not checked against any registered list.'

In [45]:
#https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/elastic-search.md